In [ ]:
import time
import math
import smbus2 as smbus
import numpy as np

class PCA9685:
    def __init__(self, address=0x40, debug=False):
        self.bus = smbus.SMBus(1)
        self.address = address
        self.debug = debug
        self.write(0x00, 0x00)

    def write(self, reg, value):
        self.bus.write_byte_data(self.address, reg, value)

    def setPWMFreq(self, freq):
        prescaleval = 25000000.0 / (4096.0 * freq) - 1.0
        prescale = math.floor(prescaleval + 0.5)
        oldmode = self.bus.read_byte_data(self.address, 0x00)
        self.write(0x00, (oldmode & 0x7F) | 0x10)
        self.write(0xFE, int(prescale))
        self.write(0x00, oldmode)
        time.sleep(0.005)
        self.write(0x00, oldmode | 0x80)

    def setPWM(self, channel, on, off):
        self.write(0x06 + 4 * channel, on & 0xFF)
        self.write(0x07 + 4 * channel, on >> 8)
        self.write(0x08 + 4 * channel, off & 0xFF)
        self.write(0x09 + 4 * channel, off >> 8)

    def setServoPulse(self, channel, pulse):
        pulse = pulse * 4096 / 20000
        self.setPWM(channel, 0, int(pulse))

# Arm segment lengths (in mm)
l1, l2, l3, l4 = 105, 265, 220, 145

# Joint calibration offsets
calibration_offsets = [90, 50, 135, 135]

def inverse_kinematics(x, y, z, beta, type="upper"):
    """
    Inverse kinematics calculation: Given x, y, z, and beta, calculate joint angles
    """
    beta = np.deg2rad(beta)
    theta_0 = np.arctan2(y, x)
    theta_3 = beta

    x_tmp = x / np.cos(theta_0) - l4 * np.cos(theta_3)
    z_tmp = z - l1 - l4 * np.sin(theta_3)
    r = np.sqrt(x_tmp**2 + z_tmp**2)

    if r > l2 + l3 or r < abs(l2 - l3):
        print("Target position is out of reach")
        return None

    cos_theta2 = (r**2 - l2**2 - l3**2) / (2 * l2 * l3)
    cos_theta2 = np.clip(cos_theta2, -1, 1)
    theta_2 = np.arccos(cos_theta2)
    if type == "upper":
        theta_2 = -theta_2

    epsilon = np.arctan2(l3 * np.sin(theta_2), l2 + l3 * np.cos(theta_2))
    phi = np.arctan2(z_tmp, x_tmp)
    theta_1 = phi - epsilon
    theta_3 = theta_3 - theta_2 - theta_1

    return np.degrees([theta_0, theta_1, theta_2, theta_3])

def angle_to_pwm(angle, min_pulse=500, max_pulse=2500, max_angle=270):
    """
    Convert angle to PWM signal
    """
    return int(min_pulse + (angle / max_angle) * (max_pulse - min_pulse))

def send_to_arm(pwm, theta_0, theta_1, theta_2, theta_3):
    """
    Send joint angles (converted to PWM) to the robotic arm
    """
    try:
        theta_0 += calibration_offsets[0]
        theta_1 = 270 - (theta_1 + calibration_offsets[1])
        theta_2 += calibration_offsets[2]
        theta_3 += calibration_offsets[3]

        theta_0 = max(0, min(180, theta_0))
        theta_1 = max(0, min(270, theta_1))
        theta_2 = max(0, min(270, theta_2))
        theta_3 = max(0, min(270, theta_3))

        pwm_0 = angle_to_pwm(theta_0, max_angle=180)
        pwm_1 = angle_to_pwm(theta_1, max_angle=270)
        pwm_2 = angle_to_pwm(theta_2, max_angle=270)
        pwm_3 = angle_to_pwm(theta_3, max_angle=270)

        pwm.setServoPulse(0, pwm_0)
        pwm.setServoPulse(1, pwm_1)
        pwm.setServoPulse(2, pwm_2)
        pwm.setServoPulse(3, pwm_3)

        print(f"Sending PWM signals: Base({pwm_0}), Shoulder({pwm_1}), Elbow({pwm_2}), Wrist({pwm_3})")

    except Exception as e:
        print(f"send_to_arm error: {e}")

def move_to_xyz(pwm, x, y, z, beta=45, type="upper"):
    """
    Control the robotic arm to move to a specified XYZ position
    """
    try:
        joint_angles = inverse_kinematics(x, y, z, beta, type=type)
        if joint_angles is not None:
            theta_0, theta_1, theta_2, theta_3 = joint_angles
            print(f"Calculated joint angles:")
            print(f"Base joint (theta_0): {theta_0:.2f}°")
            print(f"Shoulder joint (theta_1): {theta_1:.2f}°")
            print(f"Elbow joint (theta_2): {theta_2:.2f}°")
            print(f"Wrist joint (theta_3): {theta_3:.2f}°")
            
            send_to_arm(pwm, theta_0, theta_1, theta_2, theta_3)
            print(f"Successfully moved to X={x}, Y={y}, Z={z}")
        else:
            print("Cannot reach the specified position")
    except Exception as e:
        print(f"move_to_xyz error: {e}")

def test_coordinates(pwm):
    test_points = [ (-178,-86, 182, 0),]
    for x, y, z, beta in test_points:
        print(f"测试移动到 X={x}, Y={y}, Z={z}, beta={beta}°")
        move_to_xyz(pwm, x, y, z, beta)
        time.sleep(2)

def main():
    pwm = PCA9685(0x40, debug=False)
    pwm.setPWMFreq(50)
    try:
        test_coordinates(pwm)
    except KeyboardInterrupt:
        print("程序终止。")

if __name__ == "__main__":
    main()